In [1]:
import pandas as pd
import numpy as np
from numpy import inf, mean, median
import math
import datetime
import json
import os
import glob
import pickle
import requests
# import requests_cache
import plotly.graph_objects as go
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, HeatMap
import geopandas as gpd
import osm2geojson
import geojson
import h3
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from geopy.distance import geodesic, great_circle

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

# Moscow H3 Hexagons

In [3]:
h3_resolution = 8  # resolution is between 0 and 15, with 0 being coarsest and 15 being finest

In [4]:
with open('data_moscow_h3_hexes_gdf.pickle', 'rb') as _f:
    data_moscow_h3_hexes_gdf = pickle.load(_f)

In [5]:
data_moscow_h3_hexes_gdf.head()

,h3_id,h3_geo_boundary,h3_centroid
0,8811aa7567fffff,"POLYGON ((37.64593 55.89199, 37.65035 55.88806...","(55.8924532720714, 37.65386487409501)"
1,8811aa78dbfffff,"POLYGON ((37.65958 55.76722, 37.66399 55.76329...","(55.76767884611269, 37.66749214277792)"
2,8811aa7811fffff,"POLYGON ((37.61669 55.72283, 37.62110 55.71891...","(55.72329727072804, 37.62458939893891)"
3,881181b29bfffff,"POLYGON ((37.55553 55.52243, 37.55992 55.51850...","(55.52289410540872, 37.563385879195096)"
4,8811aa7a39fffff,"POLYGON ((37.53236 55.73040, 37.53677 55.72648...","(55.730869945708, 37.540257976066876)"


In [6]:
data_moscow_h3_hexes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   h3_id            1464 non-null   object  
 1   h3_geo_boundary  1464 non-null   geometry
 2   h3_centroid      1464 non-null   object  
dtypes: geometry(1), object(2)
memory usage: 45.8+ KB


# Accidents With Injured

## Load Accidents Data

In [ ]:
# выдает ошибку, где-то координаты null

# with open('data_moscow_accidents_injured.geojson', 'r') as _f:
#     data_moscow_accidents_gj = geojson.load(_f)

In [10]:
with open('data_moscow_accidents_injured.geojson', 'r') as _f:
    data_moscow_accidents_gdf = gpd.read_file(_f)

In [11]:
data_moscow_accidents_gdf.head()

,id,light,point,region,address,category,datetime,severity,dead_count,injured_count,parent_region,participants_count,geometry
0,1527476,"В темное время суток, освещение включено","{'lat': 55.6657, 'long': 37.5411}",Ломоносовский,"г Москва, ул Архитектора Власова, 49а",Наезд на пешехода,2018-02-13T20:05:00,Легкий,0,1,Москва,2,POINT (37.54110 55.66570)
1,1527477,Светлое время суток,"{'lat': 55.6823, 'long': 37.5366}",Ломоносовский,"г Москва, пр-кт Ленинский, 78",Наезд на пешехода,2018-02-10T15:25:00,Тяжёлый,0,1,Москва,2,POINT (37.53660 55.68230)
2,1527478,Сумерки,"{'lat': 55.8089, 'long': 37.7817}",Северное Измайлово,"г Москва, ш Щёлковское, 28 56",Наезд на пешехода,2018-02-12T17:25:00,Тяжёлый,0,1,Москва,2,POINT (37.78170 55.80890)
3,1527479,"В темное время суток, освещение включено","{'lat': 55.4836, 'long': 37.48}",Северное Измайлово,"г Москва, ш Щёлковское, 56 72",Наезд на пешехода,2018-02-27T19:55:00,Легкий,0,1,Москва,2,POINT (37.48000 55.48360)
4,1527480,"В темное время суток, освещение включено","{'lat': 55.6775, 'long': 37.5297}",Ломоносовский,"г Москва, пр-кт Ленинский, 86",Столкновение,2018-02-08T19:15:00,Тяжёлый,0,1,Москва,3,POINT (37.52970 55.67750)


In [12]:
data_moscow_accidents_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 55769 entries, 0 to 55768
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   id                  55769 non-null  int64   
 1   light               55769 non-null  object  
 2   point               55769 non-null  object  
 3   region              55769 non-null  object  
 4   address             52746 non-null  object  
 5   category            55769 non-null  object  
 6   datetime            55769 non-null  object  
 7   severity            55769 non-null  object  
 8   dead_count          55769 non-null  int64   
 9   injured_count       55769 non-null  int64   
 10  parent_region       55769 non-null  object  
 11  participants_count  55769 non-null  int64   
 12  geometry            55686 non-null  geometry
dtypes: geometry(1), int64(4), object(8)
memory usage: 5.5+ MB


In [13]:
data_moscow_accidents_gdf = data_moscow_accidents_gdf.loc[pd.notnull(data_moscow_accidents_gdf['geometry'])].reset_index(drop=True)

In [14]:
data_moscow_accidents_gdf['lon'] = data_moscow_accidents_gdf['geometry'].x
data_moscow_accidents_gdf['lat'] = data_moscow_accidents_gdf['geometry'].y

In [15]:
data_moscow_accidents_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 55686 entries, 0 to 55685
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   id                  55686 non-null  int64   
 1   light               55686 non-null  object  
 2   point               55686 non-null  object  
 3   region              55686 non-null  object  
 4   address             52677 non-null  object  
 5   category            55686 non-null  object  
 6   datetime            55686 non-null  object  
 7   severity            55686 non-null  object  
 8   dead_count          55686 non-null  int64   
 9   injured_count       55686 non-null  int64   
 10  parent_region       55686 non-null  object  
 11  participants_count  55686 non-null  int64   
 12  geometry            55686 non-null  geometry
 13  lon                 55686 non-null  float64 
 14  lat                 55686 non-null  float64 
dtypes: float64(2), geometry(1), 

## Accidents' Hexagons

In [16]:
%%time
h3_ids = []
h3_checks = []

list_data_moscow_h3_id = list(data_moscow_h3_hexes_gdf['h3_id'])

for index, row in data_moscow_accidents_gdf.iterrows():
    h3_id = h3.geo_to_h3(lat = row['lat'], 
                         lng = row['lon'], 
                         resolution = h3_resolution)
    h3_ids.append(h3_id)

    if h3_id in list_data_moscow_h3_id:
        h3_checks.append(True)
    else:
        h3_checks.append(False)

CPU times: user 11.9 s, sys: 59.9 ms, total: 11.9 s
Wall time: 11.9 s


In [17]:
data_moscow_accidents_gdf['h3_id'] = h3_ids
data_moscow_accidents_gdf['h3_check'] = h3_checks

In [19]:
data_moscow_accidents_gdf.head().T

,0,1,2,3,4
id,1527476,1527477,1527478,1527479,1527480
light,"В темное время суток, освещение включено",Светлое время суток,Сумерки,"В темное время суток, освещение включено","В темное время суток, освещение включено"
point,"{'lat': 55.6657, 'long': 37.5411}","{'lat': 55.6823, 'long': 37.5366}","{'lat': 55.8089, 'long': 37.7817}","{'lat': 55.4836, 'long': 37.48}","{'lat': 55.6775, 'long': 37.5297}"
region,Ломоносовский,Ломоносовский,Северное Измайлово,Северное Измайлово,Ломоносовский
address,"г Москва, ул Архитектора Власова, 49а","г Москва, пр-кт Ленинский, 78","г Москва, ш Щёлковское, 28 56","г Москва, ш Щёлковское, 56 72","г Москва, пр-кт Ленинский, 86"
category,Наезд на пешехода,Наезд на пешехода,Наезд на пешехода,Наезд на пешехода,Столкновение
datetime,2018-02-13T20:05:00,2018-02-10T15:25:00,2018-02-12T17:25:00,2018-02-27T19:55:00,2018-02-08T19:15:00
severity,Легкий,Тяжёлый,Тяжёлый,Легкий,Тяжёлый
dead_count,0,0,0,0,0
injured_count,1,1,1,1,1


In [20]:
data_moscow_accidents_gdf.groupby('h3_check').size()

h3_check
False     9798
True     45888
dtype: int64

In [21]:
with open('data_moscow_accidents_gdf.pickle', 'wb') as _f:
    pickle.dump(data_moscow_accidents_gdf, _f, pickle.HIGHEST_PROTOCOL)

# Update Moscow H3 Hexagons

## Accidents With Injured

In [ ]:
with open('data_moscow_accidents_gdf.pickle', 'rb') as _f:
    data_moscow_accidents_gdf = pickle.load(_f)

In [22]:
data_moscow_accidents_gr = data_moscow_accidents_gdf[['h3_id']].groupby('h3_id').size().reset_index(name='count')

In [24]:
data_moscow_accidents_gr.head()

,h3_id,count
0,8800ec5629fffff,1
1,8803240355fffff,1
2,88032411b3fffff,1
3,880ab23401fffff,1
4,881089a603fffff,1


In [25]:
data_moscow_accidents_gr.groupby('count').size()

count
1      1157
2       297
3       158
4       103
5        74
6        87
7        67
8        61
9        49
10       43
11       43
12       37
13       34
14       31
15       32
16       27
17       34
18       29
19       31
20       25
21       20
22       23
23       35
24       16
25       21
26       22
27       23
28       25
29       29
30       17
31       24
32       24
33       32
34       17
35       26
36       22
37       18
38       23
39       24
40       15
41       20
42       19
43       24
44       18
45       19
46       23
47       17
48       17
49       22
50       18
51       19
52       11
53       12
54       11
55       16
56       13
57       13
58        7
59        8
60        6
61       11
62       11
63        9
64       10
65       11
66        8
67        8
68       10
69        6
70        3
71        5
72        8
73        8
74        6
75        3
76        8
77        5
78        4
79        4
80        2
81        8
82        1
83        

In [26]:
data_moscow_accidents_gr.loc[data_moscow_accidents_gr['count']>70, 'count'] = 70

In [27]:
data_moscow_h3_hexes_gdf = data_moscow_h3_hexes_gdf.merge(data_moscow_accidents_gr, right_on='h3_id', left_on='h3_id', how='left')

In [29]:
data_moscow_h3_hexes_gdf.loc[pd.isnull(data_moscow_h3_hexes_gdf['count'])].count()

h3_id              102
h3_geo_boundary    102
h3_centroid        102
count                0
dtype: int64

In [30]:
data_moscow_h3_hexes_gdf.loc[pd.isnull(data_moscow_h3_hexes_gdf['count']), 'count'] = 0

In [31]:
data_moscow_h3_hexes_gdf['count'] = data_moscow_h3_hexes_gdf['count'].astype('int32')

In [32]:
data_moscow_h3_hexes_gdf.rename(columns={'count':'accidents_injured_count'}, inplace=True)

In [33]:
data_moscow_h3_hexes_gdf.head()

,h3_id,h3_geo_boundary,h3_centroid,accidents_injured_count
0,8811aa7567fffff,"POLYGON ((37.64593 55.89199, 37.65035 55.88806...","(55.8924532720714, 37.65386487409501)",25
1,8811aa78dbfffff,"POLYGON ((37.65958 55.76722, 37.66399 55.76329...","(55.76767884611269, 37.66749214277792)",28
2,8811aa7811fffff,"POLYGON ((37.61669 55.72283, 37.62110 55.71891...","(55.72329727072804, 37.62458939893891)",68
3,881181b29bfffff,"POLYGON ((37.55553 55.52243, 37.55992 55.51850...","(55.52289410540872, 37.563385879195096)",0
4,8811aa7a39fffff,"POLYGON ((37.53236 55.73040, 37.53677 55.72648...","(55.730869945708, 37.540257976066876)",49


In [34]:
data_moscow_h3_hexes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   h3_id                    1464 non-null   object  
 1   h3_geo_boundary          1464 non-null   geometry
 2   h3_centroid              1464 non-null   object  
 3   accidents_injured_count  1464 non-null   int32   
dtypes: geometry(1), int32(1), object(2)
memory usage: 51.5+ KB


# Explore Moscow H3 Hexagons With Accidents

In [36]:
data_moscow_h3_hexes_gdf.head()

,h3_id,h3_geo_boundary,h3_centroid,accidents_injured_count
0,8811aa7567fffff,"POLYGON ((37.64593 55.89199, 37.65035 55.88806...","(55.8924532720714, 37.65386487409501)",25
1,8811aa78dbfffff,"POLYGON ((37.65958 55.76722, 37.66399 55.76329...","(55.76767884611269, 37.66749214277792)",28
2,8811aa7811fffff,"POLYGON ((37.61669 55.72283, 37.62110 55.71891...","(55.72329727072804, 37.62458939893891)",68
3,881181b29bfffff,"POLYGON ((37.55553 55.52243, 37.55992 55.51850...","(55.52289410540872, 37.563385879195096)",0
4,8811aa7a39fffff,"POLYGON ((37.53236 55.73040, 37.53677 55.72648...","(55.730869945708, 37.540257976066876)",49


In [37]:
data_moscow_h3_hexes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   h3_id                    1464 non-null   object  
 1   h3_geo_boundary          1464 non-null   geometry
 2   h3_centroid              1464 non-null   object  
 3   accidents_injured_count  1464 non-null   int32   
dtypes: geometry(1), int32(1), object(2)
memory usage: 51.5+ KB


In [38]:
fig_h3_accidents_injured = px.bar(data_moscow_h3_hexes_gdf.groupby('accidents_injured_count').size().reset_index(name='count'), x='accidents_injured_count', y='count')

In [39]:
fig_h3_accidents_injured.write_html('fig_h3_accidents_injured.html')

In [40]:
map_h3_accidents_injured = folium.Map(width=1200, height=700, zoom_start=10, location=[55.75, 37.6])

#     создаем инстанс карты с центром в выбранных координатах;
#     добавляем инстанс картограммы (choropleth);
#     в инстансе картограммы мы задаем много атрибутов:
#     geo_data — геоданные, мы конвертируем данные нашего датафрейма в geojson;
#     name — задаем имя слоя;
#     data — непосредственно данные, их мы выбираем тоже из датафрейма;
#     key_on — ключ для соединения (обратите внимание, в geojson все атрибуты сложены в отдельный элемент, properties);
#     columns — ключ и атрибут для раскрашивания;
#     fill_color, fill_opacity, line_weight, line_opacity — цветовая шкала заливки, прозрачность заливки, ширина и прозрачность линий;
#     legend_name — заголовок легенды;
#     highlight — добавление интерактива (подсветки при наведении и приближения при клике) у объектов.

map_h3_accidents_injured.choropleth(
    geo_data = data_moscow_h3_hexes_gdf[['h3_id', 'h3_geo_boundary']].to_json(),
    name = 'choropleth',
    data = data_moscow_h3_hexes_gdf[['h3_id', 'accidents_injured_count']],
    key_on = 'feature.properties.h3_id',
    columns = ['h3_id', 'accidents_injured_count'],
    fill_color = 'YlGnBu',
    color = "blue",
    line_weight = 1,
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'type',
    highlight = True
)

map_h3_accidents_injured

/home/jovyan/new_env/lib/python3.8/site-packages/folium/folium.py:409: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.



In [41]:
map_h3_accidents_injured.save("map_h3_accidents_injured.html")

### Smoothing

In [42]:
%%time

data_moscow_h3_hexes_1 = data_moscow_h3_hexes_gdf[['h3_id']].copy()
data_moscow_h3_hexes_1['hex_1'] = data_moscow_h3_hexes_1['h3_id'].apply(lambda x: list(h3.hex_ring(x,1)))

data_moscow_h3_hexes_1_ungr = pd.DataFrame([], columns=['h3_id', 'hex_1'])

for i, row in data_moscow_h3_hexes_1.iterrows():
    for h in row['hex_1']:
        data_moscow_h3_hexes_1_ungr = data_moscow_h3_hexes_1_ungr.append({'h3_id':row['h3_id'], 'hex_1':h}, ignore_index=True)

CPU times: user 31.9 s, sys: 28.1 ms, total: 31.9 s
Wall time: 31.9 s


In [44]:
data_moscow_h3_hexes_1.head()

,h3_id,hex_1
0,8811aa7567fffff,"[8811aa7561fffff, 8811aa625bfffff, 8811aa752df..."
1,8811aa78dbfffff,"[8811aa78d1fffff, 8811aa78d9fffff, 8811aa78d3f..."
2,8811aa7811fffff,"[8811aa7815fffff, 8811aa7813fffff, 8811aa7817f..."
3,881181b29bfffff,"[8811aa4da7fffff, 881181b299fffff, 881181b293f..."
4,8811aa7a39fffff,"[8811aa7a3dfffff, 8811aa7a07fffff, 8811aa7a15f..."


In [43]:
data_moscow_h3_hexes_1_ungr.head()

,h3_id,hex_1
0,8811aa7567fffff,8811aa7561fffff
1,8811aa7567fffff,8811aa625bfffff
2,8811aa7567fffff,8811aa752dfffff
3,8811aa7567fffff,8811aa7565fffff
4,8811aa7567fffff,8811aa7529fffff


In [45]:
data_moscow_h3_hexes_1_ungr.shape

(8784, 2)

In [46]:
%%time
data_moscow_h3_hexes_1_ungr = data_moscow_h3_hexes_1_ungr.merge(data_moscow_h3_hexes_gdf[['h3_id', 'accidents_injured_count']], right_on='h3_id', left_on='hex_1', how='left')

CPU times: user 11.7 ms, sys: 7.89 ms, total: 19.6 ms
Wall time: 16.4 ms


In [47]:
data_moscow_h3_hexes_1_ungr.head()

,h3_id_x,hex_1,h3_id_y,accidents_injured_count
0,8811aa7567fffff,8811aa7561fffff,8811aa7561fffff,28.0
1,8811aa7567fffff,8811aa625bfffff,8811aa625bfffff,55.0
2,8811aa7567fffff,8811aa752dfffff,8811aa752dfffff,70.0
3,8811aa7567fffff,8811aa7565fffff,8811aa7565fffff,40.0
4,8811aa7567fffff,8811aa7529fffff,NaN,NaN


In [48]:
data_moscow_h3_hexes_1_ungr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   h3_id_x                  8784 non-null   object 
 1   hex_1                    8784 non-null   object 
 2   h3_id_y                  8188 non-null   object 
 3   accidents_injured_count  8188 non-null   float64
dtypes: float64(1), object(3)
memory usage: 343.1+ KB


In [49]:
data_moscow_h3_hexes_1_ungr.dropna(inplace=True)

In [50]:
data_moscow_h3_hexes_1_ungr.rename(columns={'h3_id_x':'h3_id', 'accidents_injured_count':'accidents_injured_kRing'}, inplace=True)
data_moscow_h3_hexes_1_ungr.drop('h3_id_y', axis=1, inplace=True)

In [51]:
data_moscow_h3_hexes_1_ungr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8188 entries, 0 to 8783
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   h3_id                    8188 non-null   object 
 1   hex_1                    8188 non-null   object 
 2   accidents_injured_kRing  8188 non-null   float64
dtypes: float64(1), object(2)
memory usage: 255.9+ KB


In [52]:
data_moscow_h3_hexes_1_gr = data_moscow_h3_hexes_1_ungr[['h3_id', 'accidents_injured_kRing']].groupby('h3_id').mean().reset_index()

In [53]:
data_moscow_h3_hexes_1_gr.head()

,h3_id,accidents_injured_kRing
0,881181b28bfffff,8.0
1,881181b293fffff,12.5
2,881181b297fffff,16.5
3,881181b299fffff,3.2
4,881181b29bfffff,7.6


In [54]:
data_moscow_h3_hexes_1_gr.shape

(1463, 2)

In [55]:
data_moscow_h3_hexes_gdf = data_moscow_h3_hexes_gdf.merge(data_moscow_h3_hexes_1_gr, left_on='h3_id', right_on='h3_id', how='left')

In [56]:
data_moscow_h3_hexes_gdf.head()

,h3_id,h3_geo_boundary,h3_centroid,accidents_injured_count,accidents_injured_kRing
0,8811aa7567fffff,"POLYGON ((37.64593 55.89199, 37.65035 55.88806...","(55.8924532720714, 37.65386487409501)",25,48.250000
1,8811aa78dbfffff,"POLYGON ((37.65958 55.76722, 37.66399 55.76329...","(55.76767884611269, 37.66749214277792)",28,34.500000
2,8811aa7811fffff,"POLYGON ((37.61669 55.72283, 37.62110 55.71891...","(55.72329727072804, 37.62458939893891)",68,53.833333
3,881181b29bfffff,"POLYGON ((37.55553 55.52243, 37.55992 55.51850...","(55.52289410540872, 37.563385879195096)",0,7.600000
4,8811aa7a39fffff,"POLYGON ((37.53236 55.73040, 37.53677 55.72648...","(55.730869945708, 37.540257976066876)",49,33.666667


In [57]:
data_moscow_h3_hexes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   h3_id                    1464 non-null   object  
 1   h3_geo_boundary          1464 non-null   geometry
 2   h3_centroid              1464 non-null   object  
 3   accidents_injured_count  1464 non-null   int32   
 4   accidents_injured_kRing  1463 non-null   float64 
dtypes: float64(1), geometry(1), int32(1), object(2)
memory usage: 62.9+ KB


In [58]:
def f_smooth(score_hex_score_kRing, coef_hex):
    score_hex = score_hex_score_kRing[0]
    score_kRing = score_hex_score_kRing[1]
    if coef_hex < 0:
        return None
    elif coef_hex > 1:
        return None
    elif pd.isnull(score_kRing):
        return int(score_hex)
    else:
        return int(coef_hex*score_hex + (1-coef_hex)*score_kRing)

In [59]:
data_moscow_h3_hexes_gdf['score_smooth'] = data_moscow_h3_hexes_gdf[['accidents_injured_count', 'accidents_injured_kRing']].apply(f_smooth, axis=1, args=(0.6,))

In [60]:
data_moscow_h3_hexes_gdf.head()

,h3_id,h3_geo_boundary,h3_centroid,accidents_injured_count,accidents_injured_kRing,score_smooth
0,8811aa7567fffff,"POLYGON ((37.64593 55.89199, 37.65035 55.88806...","(55.8924532720714, 37.65386487409501)",25,48.250000,34
1,8811aa78dbfffff,"POLYGON ((37.65958 55.76722, 37.66399 55.76329...","(55.76767884611269, 37.66749214277792)",28,34.500000,30
2,8811aa7811fffff,"POLYGON ((37.61669 55.72283, 37.62110 55.71891...","(55.72329727072804, 37.62458939893891)",68,53.833333,62
3,881181b29bfffff,"POLYGON ((37.55553 55.52243, 37.55992 55.51850...","(55.52289410540872, 37.563385879195096)",0,7.600000,3
4,8811aa7a39fffff,"POLYGON ((37.53236 55.73040, 37.53677 55.72648...","(55.730869945708, 37.540257976066876)",49,33.666667,42


In [61]:
data_moscow_h3_hexes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   h3_id                    1464 non-null   object  
 1   h3_geo_boundary          1464 non-null   geometry
 2   h3_centroid              1464 non-null   object  
 3   accidents_injured_count  1464 non-null   int32   
 4   accidents_injured_kRing  1463 non-null   float64 
 5   score_smooth             1464 non-null   int64   
dtypes: float64(1), geometry(1), int32(1), int64(1), object(2)
memory usage: 74.3+ KB


In [63]:
map_h3_accidents_injured_smooth = folium.Map(width=1200, height=700, zoom_start=10, location=[55.75, 37.6])

#     создаем инстанс карты с центром в выбранных координатах;
#     добавляем инстанс картограммы (choropleth);
#     в инстансе картограммы мы задаем много атрибутов:
#     geo_data — геоданные, мы конвертируем данные нашего датафрейма в geojson;
#     name — задаем имя слоя;
#     data — непосредственно данные, их мы выбираем тоже из датафрейма;
#     key_on — ключ для соединения (обратите внимание, в geojson все атрибуты сложены в отдельный элемент, properties);
#     columns — ключ и атрибут для раскрашивания;
#     fill_color, fill_opacity, line_weight, line_opacity — цветовая шкала заливки, прозрачность заливки, ширина и прозрачность линий;
#     legend_name — заголовок легенды;
#     highlight — добавление интерактива (подсветки при наведении и приближения при клике) у объектов.

map_h3_accidents_injured_smooth.choropleth(
    geo_data = data_moscow_h3_hexes_gdf[['h3_id', 'h3_geo_boundary']].to_json(),
    name = 'choropleth',
    data = data_moscow_h3_hexes_gdf[['h3_id', 'score_smooth']],
    key_on = 'feature.properties.h3_id',
    columns = ['h3_id', 'score_smooth'],
    fill_color = 'YlGnBu',
    color = "blue",
    line_weight = 1,
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'type',
    highlight = True
)

map_h3_accidents_injured_smooth

/home/jovyan/new_env/lib/python3.8/site-packages/folium/folium.py:409: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.



In [64]:
map_h3_accidents_injured_smooth.save("map_h3_accidents_injured_smooth.html")

In [62]:
with open('data_moscow_h3_hexes_accidents_injured_gdf.pickle', 'wb') as _f:
    pickle.dump(data_moscow_h3_hexes_gdf, _f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data_moscow_h3_hexes_accidents_injured_gdf.pickle', 'rb') as _f:
    data_moscow_h3_hexes_gdf = pickle.load(_f)